In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

import time

from PyDemoMomentumopt import MotionPlanner
from momentumopt.kinoptpy.momentum_kinematics_optimizer import MomentumKinematicsOptimizer

def replay_kinematic_plan():
    for ks in motion_planner.kin_optimizer.kinematics_sequence.kinematics_states:
        q = ks.robot_posture.generalized_joint_positions
        motion_planner.kin_optimizer.robot.display(np.matrix(q).T)
        time.sleep(0.1)

In [ ]:
# cfg_file = '../config/SAB_Demos/cfg_quadruped_forward_jump.yaml'
# cfg_file = '../config/cfg_demo01_walk_short.yaml'
cfg_file = '../config/cfg_demo02_trot.yaml'
# cfg_file = '../config/cfg_quadruped_lift_rear.yaml'
# cfg_file = '../config/cfg_quadruped_lift_leg_momentum.yaml'
# cfg_file = '../config/cfg_quadruped_lift_leg_momentum_rotated.yaml'

motion_planner = MotionPlanner(cfg_file, KinOpt=MomentumKinematicsOptimizer)
inv_kin = motion_planner.kin_optimizer.inv_kin
kin_opt = motion_planner.kin_optimizer

# Dynamic optimization
motion_planner.optimize_dynamics(0)
# optimized_kin_plan, optimized_motion_eff, optimized_dyn_plan, planner_setting, time_vector = motion_planner.optimize_motion()

In [ ]:
etg = motion_planner.kin_optimizer.endeff_traj_generator
etg.z_offset = 0.05

inv_kin.w_com_tracking[:3] = 1.
inv_kin.w_com_tracking[3:] = 10.
# inv_kin.w_com_tracking[4] = 10.
inv_kin.w_endeff_contact = 1.
inv_kin.p_endeff_tracking = 1.
inv_kin.p_com_tracking = 1.
motion_planner.kin_optimizer.reg_orientation = 0.05

motion_planner.optimize_kinematics(0)
motion_planner.plot_centroidal()
motion_planner.replay_kinematics()
plt.savefig('iter_00.pdf')

In [ ]:
for i in range(3):
    motion_planner.optimize_dynamics(i)
    motion_planner.kin_optimizer.reg_orientation = 0.05

    motion_planner.optimize_kinematics(i)
    motion_planner.plot_centroidal()
    motion_planner.replay_kinematics()
    plt.savefig('iter_%02d.pdf' % (i))
    
motion_planner.save_files()

In [ ]:
dt = 0.001                      # controller time step

import pinocchio as se3
from pinocchio.utils import zero

import yaml
import numpy as np
import matplotlib.pyplot as plt

import momentumopt
from quadruped.quadruped_wrapper import QuadrupedWrapper

import time
import tsid
import os
import pinocchio as se3
import numpy.matlib as matlib
from numpy import nan
from numpy.linalg import norm as norm

np.set_printoptions(suppress=True, precision=2)

###
# Read the data files.
read_data = lambda filename: np.genfromtxt(filename)[:, 1:]

desired = {
    'q': read_data('quadruped_generalized_positions.dat'),
    'dq': read_data('quadruped_generalized_velocities.dat'),
#     'ddq': read_data('quadruped_generalized_acceleration.dat'),
    'forces': read_data('quadruped_forces.dat').reshape(-1, 4, 3),
}
desired['ddq'] = np.zeros_like(desired['dq'])
desired['ddq'][:-1] = (desired['dq'][1:] - desired['dq'][:-1])/dt


def base_pos_vec(q):
    """ Get the xyzrpy vector from the 7 dimensional xyz quaternion vector. """
    arr = q[3:7]
    quad = se3.Quaternion(arr[3], arr[0], arr[1], arr[2])

    return np.vstack([
        q[:3].reshape(-1, 1),
        se3.utils.matrixToRpy(quad.matrix())
    ])

desired['q'][:, 2] += 0.32

###
# Load the URDF model
urdf_path = '../src/urdf/quadruped.urdf'
urdf_model_path = '../src/urdf/'
robot_display = QuadrupedWrapper(urdf_path)

###
# Setup model weights and connect to display.

mu = 0.8                            # friction coefficient
fMin = 0.0                          # minimum normal force
fMax = 100.0                       # maximum normal force
contact_frames = ['BR_contact', 'BL_contact', 'FR_contact', 'FL_contact']
contactNormal = np.matrix([0., 0., 1.]).T   # direction of the normal to the contact surface

w_com = 1.0                     # weight of center of mass task
w_posture = 1e2                # weight of joint posture task
w_base = 10.0                    # weight of joint posture task
w_forceRef = 1e0               # weight of force regularization task
w_effContact = 50.
w_effRef = 1.0

kp_contact = 100.0               # proportional gain of contact constraint
kp_com = 10.0                   # proportional gain of center of mass task
kp_posture = 0.1               # proportional gain of joint posture task
kp_base = 50.0                  # proportional gain of joint base task
kp_eff = 100.0                   # proportional gain of joint base task

PRINT_N = 500                   # print every PRINT_N time steps
DISPLAY_N = 25                  # update robot configuration in viwewer every DISPLAY_N time steps

In [ ]:
path = urdf_path
urdf = urdf_path
vector = se3.StdVec_StdString()
vector.extend(item for item in urdf_model_path)
robot = tsid.RobotWrapper(urdf, vector, se3.JointModelFreeFlyer(), False)

# for gepetto viewer
#q = se3.getNeutralConfigurationFromSrdf(robot.model(), srdf, False)
q = desired['q'][0].reshape(-1, 1)
v = matlib.zeros(robot.nv).T

robot_display.display(q)

###
# Add tasks for the invdyn problem.
assert [robot.model().existFrame(name) for name in contact_frames]

t = 0.0                         # time
invdyn = tsid.InverseDynamicsFormulationAccForce("tsid", robot, False)
invdyn.computeProblemData(t, q, v)
data = invdyn.data()

# Place the robot onto the ground.
id_contact = robot_display.model.getFrameId(contact_frames[0])
# q[2] -= robot.framePosition(data, id_contact).translation[2, 0]
robot.computeAllTerms(data, q, v)

class Endeffector(object):
    def __init__(self, name, invdyn, robot, robot_display):
        self.invdyn = invdyn
        self.name = name
        self.robot = robot
        self.robot_display = robot_display
        self.frameId = robot.model().getFrameId(name)

        self.contact = tsid.ContactPoint(name, robot, name, contactNormal, mu, fMin, fMax)
        self.contact.setKp(kp_contact * matlib.ones(6).T)
        self.contact.setKd(2.0 * np.sqrt(kp_contact) * matlib.ones(6).T)
        self.contact.useLocalFrame(False)
        
        self.ref = tsid.TaskSE3Equality(name + '_ref', robot, self.name)
        self.ref.setKp(kp_eff * matlib.ones(6).T)
        self.ref.setKd(2.0 * np.sqrt(kp_eff) * matlib.ones(6).T)
        self.ref.useLocalFrame(False)
        self.ref.setMask(np.matrix([1, 1, 1, 0, 0, 0]).T)
        
#         base = tsid.TrajectorySE3Constant(name + "_pos")
        base = tsid.TrajectorySE3Constant(name + "_pos", robot_display.data.oMf[self.frameId])
        self.base = base
        self.sample = base.computeNext()
        
    def get_world_oriented_frame_jacobian(self, q):
        jac = se3.frameJacobian(self.robot_display.model, self.robot_display.data, 
                                q, self.frameId, se3.ReferenceFrame.LOCAL)
        world_R_joint = se3.SE3(self.robot_display.data.oMf[self.frameId].rotation, zero(3))
        return world_R_joint.action.dot(jac)
        
    def update(self, q, dq):
        # Compute the position and velocity of the endeffector.
        if self.contact_active:
            self.contact.setReference(self.robot_display.data.oMf[self.frameId])
        else:
            pos = zero(12) # The position is 3 translation + 9 rotation
            pos[:3] = self.robot_display.data.oMf[self.frameId].translation
            self.sample.pos(pos)
            self.sample.vel(self.get_world_oriented_frame_jacobian(q).dot(dq))
            self.ref.setReference(self.sample)
            
    def updateContact(self, i, forces, q, dq):
        if forces[2] > 0.5:
            if not self.contact_active:
                print(i)
                self.add_contact()
            self.contact.setForceReference(forces.reshape(-1, 1))
        else:
            if self.contact_active:
                print(i)
                self.remove_contact()
                
        self.update(q, dq)
            
    def remove_contact(self):
        print('Endeff {}: Remove contact'.format(self.name))
        self.invdyn.removeRigidContact(self.name, 0.)
        self.invdyn.addMotionTask(self.ref, w_effRef, 1, 0.)
        self.contact_active = False
        
    def add_contact(self):
        print('Endeff {}: Add contact'.format(self.name))
        invdyn.removeTask(self.name + '_ref', 0.)
        invdyn.addRigidContact(self.contact, w_forceRef, w_effContact, 1)
        self.contact_active = True


effs = 4*[None]
for i, name in enumerate(contact_frames):
    effs[i] = Endeffector(name, invdyn, robot, robot_display.robot)

# Adding tracking of the posture and base.
postureTask = tsid.TaskJointPosture("task-posture", robot)
postureTask.setKp(kp_posture * matlib.ones(robot.nv-6).T)
postureTask.setKd(2.0 * np.sqrt(kp_posture) * matlib.ones(robot.nv-6).T)
invdyn.addMotionTask(postureTask, w_posture, 1, 0.0)

baseTask = tsid.TaskSE3Equality("task-base", robot, "base_link")
baseKp = kp_base * matlib.ones(6).T
baseTask.setKp(baseKp)
baseKd = 2.0 * np.sqrt(kp_base) * matlib.ones(6).T
baseKd[3:] *= 10
baseTask.setKd(baseKd)
# baseTask.useLocalFrame(False)
invdyn.addMotionTask(baseTask, w_base, 1, 0.0)

###
# Solve the invdyn problem and simulate the found ddq solution forward.

# Tracking the motion from the trajectory optimizer.
N_SIMULATION = desired['q'].shape[0]             # number of time steps simulated

trajBase = tsid.TrajectoryEuclidianConstant("traj_base", q[:6])
sampleBase = trajBase.computeNext()
trajPosture = tsid.TrajectoryEuclidianConstant("traj_joint", q[7:])
samplePosture = trajPosture.computeNext()

solver = tsid.SolverHQuadProgFast("qp solver")
solver.resize(invdyn.nVar, invdyn.nEq, invdyn.nIn)

In [ ]:
N_SIMULATION = 2500

q = desired['q'][0].reshape(-1, 1)
v = matlib.zeros(robot.nv).T
traj_q = np.zeros((N_SIMULATION, robot.nq))
traj_dq = np.zeros((N_SIMULATION, robot.nv))
traj_tau = np.zeros((N_SIMULATION, robot.nv - 6))

for i in range(0, N_SIMULATION):
    traj_q[i] = q.copy().T
    traj_dq[i] = v.copy().T
    time_start = time.time()
    
    sampleBase.pos(base_pos_vec(desired['q'][i]))
    sampleBase.vel(desired['dq'][i, :6].reshape(-1, 1))
    sampleBase.acc(desired['ddq'][i, :6].reshape(-1, 1))
    
    samplePosture.pos(desired['q'][i, 7:].reshape(-1, 1))
    samplePosture.vel(desired['dq'][i, 6:].reshape(-1, 1))
    samplePosture.acc(desired['ddq'][i, 6:].reshape(-1, 1))
    
    baseTask.setReference(sampleBase)
    postureTask.setReference(samplePosture)

    # Update the endeffector state. Use the desired state to get
    # proper tracking information.
    q_des = desired['q'][i].reshape(-1, 1)
    dq_des = desired['dq'][i].reshape(-1, 1)
    robot_display.robot.forwardKinematics(q_des, dq_des)
    robot_display.robot.framesForwardKinematics(q_des)
    
    for k, eff in enumerate(effs):
        if i == 0:
            eff.add_contact()
        
        eff.updateContact(i, desired['forces'][i, k], q_des, dq_des)

    HQPData = invdyn.computeProblemData(t, q, v)
    if i == 0: HQPData.print_all()

    sol = solver.solve(HQPData)
    if(sol.status!=0):
        print "[%d] QP problem could not be solved! Error code:"%(i), sol.status
        break
    
    tau = invdyn.getActuatorForces(sol)
    dv = invdyn.getAccelerations(sol)
    
    traj_tau[i] = tau.T
    
    if np.any(np.isnan(q)):
        print('Found nan in q, i=', i)
        break
    
#     if i%PRINT_N == 0:
#         print "Time %.3f"%(t)
#         print "\tNormal forces: ",
#         for contact in contacts:
#             if invdyn.checkContact(contact.name, sol):
#                 f = invdyn.getContactForce(contact.name, sol)
#                 print "%4.1f"%(contact.getNormalForce(f)),

# #         print "\n\ttracking err %s: %.3f"%(comTask.name.ljust(20,'.'),       norm(comTask.position_error, 2))
#         print "\t||v||: %.3f\t ||dv||: %.3f"%(norm(v, 2), norm(dv))

    v_mean = v + 0.5*dt*dv
    v += dt*dv
    q = se3.integrate(robot.model(), q, dt*v_mean)
    t += dt
    
    if i%DISPLAY_N == 0: robot_display.display(q)

# robot_display.display(desired['q'][i].reshape(-1, 1))

plt.plot(traj_q[:, 1])
plt.plot(desired['q'][:, 1])
plt.plot(traj_tau)

In [ ]:
for q in traj_q:
    robot_display.display(q.reshape(-1, 1))

In [ ]:
for q in desired['q'][:N_SIMULATION]:
    robot_display.display(q.reshape(-1, 1))